<img src="https://github.com/insaid2018/Term-1/blob/master/Images/INSAID_Full%20Logo.png?raw=true" width="240" height="360" />

# STACKING
<img src = 'https://raw.githubusercontent.com/insaid2018/Term-4/master/images/anomaly/stacking.png'>

## Table of Content

1. [Introduction to Stacking](#section1)<br>
    - 1.1 [Method](#section101)<br>
    - 1.2 [How Stacking works?](#section102)<br>
2. [Use Case : Wine Data](#section2)<br>
3. [Data loading and description](#section3)<br>
4. [Feature Engineering](#section4)<br>
5. [Training and Prediction](#section5)<br>
6. [Conclusion](#section6)

<a id=section1></a>
### Introduction
<img src = 'https://raw.githubusercontent.com/insaid2018/Term-4/master/images/anomaly/stacking1.png'>

Ensemble methods are an excellent way to __improve predictive performance__ on the machine learning problems.<br><br>
__Stacked Generalization or stacking__ is an ensemble technique that uses a __new model__ to learn how to __best combine the predictions__ from two or more models trained on the dataset.



Stacking is a __model ensembling technique__ used to __combine__ information from __multiple__ predictive models to generate a __new model__. Often times the stacked model ( also called __2nd level model__) will outperform each of the individual models due to its something __nature and ability__ to __highlight__ each base model where it performs best and discredit each base model where it performs poorly.<br><br> For this reason, stacking is __most effective__ when the __base models are significantly different__.
<img src = 'https://raw.githubusercontent.com/insaid2018/Term-4/master/images/anomaly/stacking2.png'>

### Method

It is a different way of combining multiple models, that introduces the __concept of meta learner__.<br> Although an attractive idea, it is __less widely used than bagging and boosting__.<br> Unlike bagging and boosting, stacking may be used to __combine models of different types__.
The procedure is as follows:
1. Split the training set into **two disjoint sets**.
2. **Train** several **base learners** on the first part.
3. **Test** the base learners on the **second part**.
4. Using the **prediction from (3) as the inputs**, and the correct responses as the outputs, train a higher level learner.

### How Stacking works?

An ensemble technique that uses **predictions from multiple models** to build a model. This model is used for making predictions on the test set.<br>Below is the step-wise explanation for a simple stacked ensemble:

**1. The train set split into 10 parts**.<br>

<img src = 'https://raw.githubusercontent.com/insaid2018/Term-4/master/images/anomaly/stacking3.png'>

**2. A base model is fitted on 9 parts and predictions are made for the 10th part. This is done for each part of this train set**.

<img src = 'https://raw.githubusercontent.com/insaid2018/Term-4/master/images/anomaly/stacking4.png'>

**3. The base model is then fitted on the whole train dataset.**<br>
**4. Using this model, predictions are made on the test set.**

<img src = 'https://raw.githubusercontent.com/insaid2018/Term-4/master/images/anomaly/stacking5.png'>

**5. Step 2 to 4 are repeated for another base model resulting in another set of predictions for the train set and test set.**

<img src = 'https://raw.githubusercontent.com/insaid2018/Term-4/master/images/anomaly/stacking6.png'>

**6. The predictions from the trains et are used as features to build a new model.**

<img src = 'https://raw.githubusercontent.com/insaid2018/Term-4/master/images/anomaly/stacking7.png'>

**7. This model is used to make final predictions on the test prediction set.**

### Use case: Wine Data

<img src = 'https://raw.githubusercontent.com/insaid2018/Term-4/master/images/wine.jpg'>
- These data are the **results of a chemical analysis** of wines grown in the same region in Italy but derived from **three different cultivars.**
- The analysis determined the quantities of **13 constituents** found in each of the three types of wines.
- There are **13 variables** included in the dataset.


1. Alcohol
2. Malic acid
3. Ash
4. Alkalinity of ash
5. Magnesium
6. Total phenols
7. Flavanoids
8. Nonflavanoid phenols
9. Proanthocyanins
10. Color intensity
11. Hue
12. OD280/OD315 of diluted wines
13. Proline

<a id=section3></a>
### Importing the Packages

In [ ]:
# !pip install xgboost
# !pip install vecstack

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from vecstack import stacking

### Importing data

In [7]:
link = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data'
names = ['Class', 'Alcohol', 'Malic acid', 'Ash',
         'Alcalinity of ash' ,'Magnesium', 'Total phenols',
         'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue', 'OD280/OD315 of diluted wines',
         'Proline']

### Reading the data into Python environment

In [8]:
df = pd.read_csv(link, header=None, names=names)
df.sample(5)

,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
142,3,13.52,3.17,2.72,23.5,97,1.55,0.52,0.50,0.55,4.35,0.89,2.06,520
21,1,12.93,3.80,2.65,18.6,102,2.41,2.41,0.25,1.98,4.50,1.03,3.52,770
131,3,12.88,2.99,2.40,20.0,104,1.30,1.22,0.24,0.83,5.40,0.74,1.42,530
111,2,12.52,2.43,2.17,21.0,88,2.55,2.27,0.26,1.22,2.00,0.90,2.78,325
19,1,13.64,3.10,2.56,15.2,116,2.70,3.03,0.17,1.66,5.10,0.96,3.36,845


### Train Test Split

In [9]:
y = df[['Class']]
X = df.iloc[:,1:]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Instantiate different models

In [10]:
models = [KNeighborsClassifier(n_neighbors=5, n_jobs=-1),
         RandomForestClassifier(random_state=0, n_jobs=-1, n_estimators=100, max_depth=3),
         XGBClassifier(random_state=0, n_jobs=-1, laerning_rate=0.1, n_estimators=100, max_depth=3)
         ]

### Making first level predictions

In [11]:
S_train, S_test = stacking(models,
                           X_train,
                           y_train,
                           X_test,
                           regression=False,
                           mode='oof_pred_bag',
                           needs_proba=False,
                           save_dir=None,
                           metric=accuracy_score,
                           n_folds=4,
                           stratified=True,
                           shuffle=True,
                           random_state=0,
                           verbose=2)

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


task:         [classification]
n_classes:    [3]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [KNeighborsClassifier]
    fold  0:  [0.72972973]
    fold  1:  [0.61111111]
    fold  2:  [0.62857143]
    fold  3:  [0.76470588]
    ----
    MEAN:     [0.68352954] + [0.06517070]
    FULL:     [0.68309859]

model  1:     [RandomForestClassifier]
    fold  0:  [1.00000000]
    fold  1:  [1.00000000]
    fold  2:  [0.94285714]
    fold  3:  [1.00000000]
    ----
    MEAN:     [0.98571429] + [0.02474358]
    FULL:     [0.98591549]

model  2:     [XGBClassifier]
    fold  0:  [1.00000000]
    fold  1:  [0.97222222]
    fold  2:  [0.91428571]
    fold  3:  [0.97058824]
    ----
    MEAN:     [0.96427404] + [0.03113768]
    FULL:     [0.96478873]



In [12]:
model = XGBClassifier(random_state=0, n_jobs=-1, learning_rate=0.1,
                     n_estimators=100, max_depth=3)

### Fit the model

In [13]:
model = model.fit(S_train, y_train)

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/nbuser/anaconda3_501/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [15]:
y_pred = model.predict(S_test)
y_pred

array([1, 3, 2, 1, 2, 2, 1, 3, 2, 2, 3, 3, 1, 2, 3, 2, 1, 1, 3, 1, 1, 1,
       1, 2, 2, 2, 2, 2, 2, 3, 1, 1, 2, 1, 1, 1])

### Accuracy

In [16]:
print('Final prediction score: [%.8f]' % accuracy_score(y_test, y_pred))

Final prediction score: [0.94444444]
